<a href="https://colab.research.google.com/github/antlobach/pytorch-stuff/blob/main/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch

In [4]:
torch.__version__

'2.10.0+cu128'

In [5]:
torch.cuda.is_available()

True

In [7]:
t1 = torch.tensor(1)

In [8]:
t1

tensor(1)

In [12]:
t1.unsqueeze(0).unsqueeze(0).unsqueeze(0).shape

torch.Size([1, 1, 1])